In [15]:
%load_ext autoreload

In [20]:
%autoreload 2

In [1]:
from src.parsers import mimic, hirid
from src.modeling import discovery, plots, querier
from src.utils import constants

In [2]:
import sys
# import win32com.client
import os
import pandas as pd

def setup_io_config(root_path):
    """
    Input - Output config. Add dataset paths
    :root_path -> Repo path which contains 'data' and 'res' folders
    """

    # MIMIC
    is_shortcut = True if "data.lnk" in os.listdir(root_path) else False 
    
    if (is_shortcut):
        path_shortcut =  os.path.join(root_path, "data.lnk")
        shell = win32com.client.Dispatch("WScript.Shell")
        mimic_data = shell.CreateShortCut(path_shortcut).Targetpath
    else:
        mimic_data = os.path.join(f"{root_path}", "data") 
    mimic_path = os.path.join(f"{root_path}", "results")

    # HIRID
    hirid_data = f'{root_path}/data/hirid-a-high-time-resolution-icu-dataset-1.1.1/raw_stage/'
    hirid_path = f'{root_path}/data/hirid-a-high-time-resolution-icu-dataset-1.1.1'
    
    return mimic_data, mimic_path, hirid_data, hirid_path

def setup_stratification_config():
    gender="MF"
    age_b=40
    age_a=80 
    ethnicity="WHITE" 
    lab_mapping= constants.LAB_MAPPING
    b_w = [(0,6), (6,12)]
    a_w = [(0,4), (4,8), (8,12)]
    before_windows = b_w
    after_windows = a_w
    return gender, age_a, age_b, ethnicity, lab_mapping, before_windows, after_windows

In [171]:
# IO Config
# root_path ="C:\\Users\\danco\\My Drive\\Master\\Datasets\\MIMIC iii"
root_path = "/Users/pavan/Library/CloudStorage/GoogleDrive-f20190038@hyderabad.bits-pilani.ac.in/My Drive/TAU/Code/DrugLab"
data, res, raw_path, res_path = setup_io_config(root_path=root_path)

# Stratification Config
gender, age_a, age_b, ethnicity, lab_mapping, before_windows, after_windows = setup_stratification_config()

In [18]:
med_lab_pairs = pd.read_csv(os.path.join(res, "before_after_windows_main_med_lab_trends_first_val_40-80_MF_WHITE_doc_eval_win_with_no_vital_signs.csv")).drop(columns=["Unnamed: 0"])

/var/folders/by/jbk3mn054mv4x7_28jxcwvx80000gn/T/ipykernel_2057/3303238665.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  med_lab_pairs = pd.read_csv(os.path.join(res, "before_after_windows_main_med_lab_trends_first_val_40-80_MF_WHITE_doc_eval_win_with_no_vital_signs.csv")).drop(columns=["Unnamed: 0"])


In [20]:
med_lab_pairs

,SUBJECT_ID,OldITEMID,HADM_ID,ROW_ID_x,ROW_ID_x.1,ICUSTAY_ID,STARTTIME,ENDTIME,AMOUNT,AMOUNTUOM,...,"before_abs_(0, 6)_sp","before_time_(6, 12)_sp","before_abs_(6, 12)_sp","after_time_(0, 4)_sp","after_abs_(0, 4)_sp","after_time_(4, 8)_sp","after_abs_(4, 8)_sp","after_time_(8, 12)_sp","after_abs_(8, 12)_sp",LAB_NAME
0,63024,225975,144024,3345728,36880,296980.0,2188-06-19 17:31:00,2188-06-19 17:32:00,1.000000,dose,...,8.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Calcium
1,63024,225975,144024,3345728,36880,296980.0,2188-06-19 17:31:00,2188-06-19 17:32:00,1.000000,dose,...,0.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Creatinine
2,63024,225975,144024,3345728,36880,296980.0,2188-06-19 17:31:00,2188-06-19 17:32:00,1.000000,dose,...,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Glucose
3,63024,225975,144024,3345728,36880,296980.0,2188-06-19 17:31:00,2188-06-19 17:32:00,1.000000,dose,...,40.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hematocrit
4,63024,225975,144024,3345728,36880,296980.0,2188-06-19 17:31:00,2188-06-19 17:32:00,1.000000,dose,...,12.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hemoglobin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1826725,93321,225930,115396,3599677,44773,248745.0,2128-09-14 15:33:00,2128-09-14 18:43:00,31.666667,ml,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.033333,34.90,PTT
1826726,93321,225930,115396,3599677,44773,248745.0,2128-09-14 15:33:00,2128-09-14 18:43:00,31.666667,ml,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.033333,3.70,Potassium
1826727,93321,225930,115396,3599677,44773,248745.0,2128-09-14 15:33:00,2128-09-14 18:43:00,31.666667,ml,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.033333,1.00,Prothrombin time INR
1826728,93321,225930,115396,3599677,44773,248745.0,2128-09-14 15:33:00,2128-09-14 18:43:00,31.666667,ml,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.033333,2.53,Red blood cell


In [62]:
t_m = med_lab_pairs[['AMOUNT', 'AMOUNTUOM', 'RATE',
       'RATEUOM', 'TOTALAMOUNT', 'TOTALAMOUNTUOM','MED_NAME', 'LAB_NAME']]

In [68]:
print(list(t_m.MED_NAME.value_counts().keys()))

['NaCl 0.9%', 'Dextrose 5%', 'PO Intake', 'Solution', 'Magnesium Sulfate', 'Potassium Chloride', 'Insulin - Regular', 'Heparin Sodium (Prophylaxis)', 'Propofol', 'Vancomycin', 'Magnesium Sulfate (Bolus)', 'LR', 'Calcium Gluconate', 'Piggyback', 'OR Crystalloid Intake', 'Morphine Sulfate', 'Furosemide (Lasix)', 'KCL (Bolus)', 'Pantoprazole (Protonix)', 'Fentanyl', 'Gastric Meds', 'Phenylephrine', 'Packed Red Blood Cells', 'Pre-Admission Intake', 'Famotidine (Pepcid)', 'Metoprolol', 'Midazolam (Versed)', 'Cefazolin', 'GT Flush', 'Hydromorphone (Dilaudid)', 'Insulin - Humalog', 'Insulin - Glargine', 'Lorazepam (Ativan)', 'Piperacillin/Tazobactam (Zosyn)', 'Nitroglycerin', 'Norepinephrine', 'Heparin Sodium', 'Albumin 5%', 'D5 1/2NS', 'Sterile Water', 'OR Cell Saver Intake', 'K Phos', 'Hydralazine', 'Metronidazole', 'Fentanyl (Concentrate)', 'Ciprofloxacin', 'Cefepime', 'OR Packed RBC Intake', 'Ranitidine (Prophylaxis)', 'Fresh Frozen Plasma', 'Sodium Bicarbonate 8.4%', 'Ceftriaxone', 'Plat

In [59]:
med_lab_pairs.LAB_NAME.value_counts()

Glucose                      136383
Potassium                    132462
Sodium                       132389
Hematocrit                   131418
Creatinine                   131200
Hemoglobin                   129702
Platelets                    129107
Red blood cell               127479
Magnesium                    111646
Prothrombin time INR         103296
PTT                          102882
Calcium                      101521
Calcium ionized               73358
Lactic acid                   71455
Alanine aminotransferase      46567
Bilirubin                     46516
Asparate aminotransferase     46500
Albumin                       32594
Lipase                        19739
Amylase                       11970
Hemoglobin percent             3931
Vitamin B12                    2028
Uric Acid                      1814
Uric Acid, Urine                308
Magnesium, Urine                242
Creatinine ascites              106
Prolactin                        84
Hemoglobin C                

In [28]:
for b_w in before_windows:
    for a_w in after_windows:
        print(b_w, a_w)
        med_lab_pairs[f"ratio_{b_w}_{a_w}"] = med_lab_pairs[f"after_abs_{a_w}_sp"] / med_lab_pairs[f"before_abs_{b_w}_sp"]

(0, 6) (0, 4)
(0, 6) (4, 8)
(0, 6) (8, 12)
(6, 12) (0, 4)
(6, 12) (4, 8)
(6, 12) (8, 12)


In [100]:
# Original size of med lab pairs - 1826730
for a_w in after_windows:
    for b_w in before_windows:
        med_lab_pairs = pd.concat([ med_lab_pairs[(med_lab_pairs[f"after_time_{a_w}_sp"]<1) & (med_lab_pairs[f"ratio_{b_w}_{a_w}"]!=1)], med_lab_pairs[(med_lab_pairs[f"after_time_{a_w}_sp"]>=1) | (med_lab_pairs[f"after_time_{a_w}_sp"].isna())] ])

In [101]:
med_lab_pairs

,SUBJECT_ID,OldITEMID,HADM_ID,ROW_ID_x,ROW_ID_x.1,ICUSTAY_ID,STARTTIME,ENDTIME,AMOUNT,AMOUNTUOM,...,"after_abs_(4, 8)_sp","after_time_(8, 12)_sp","after_abs_(8, 12)_sp",LAB_NAME,"ratio_(0, 6)_(0, 4)","ratio_(0, 6)_(4, 8)","ratio_(0, 6)_(8, 12)","ratio_(6, 12)_(0, 4)","ratio_(6, 12)_(4, 8)","ratio_(6, 12)_(8, 12)"
110,70686,221833,191526,871849,38932,288251.0,2101-08-10 22:15:00,2101-08-10 22:16:00,0.500000,mg,...,NaN,NaN,NaN,Glucose,1.054795,NaN,NaN,NaN,NaN,NaN
208,70686,226089,191526,871858,38932,288251.0,2101-08-11 00:27:00,2101-08-11 00:29:00,2.000000,ml,...,NaN,NaN,NaN,Calcium,0.977273,NaN,NaN,NaN,NaN,NaN
210,70686,226089,191526,871858,38932,288251.0,2101-08-11 00:27:00,2101-08-11 00:29:00,2.000000,ml,...,156.0,NaN,NaN,Glucose,0.857143,0.675325,NaN,NaN,NaN,NaN
211,70686,226089,191526,871858,38932,288251.0,2101-08-11 00:27:00,2101-08-11 00:29:00,2.000000,ml,...,NaN,NaN,NaN,Hematocrit,0.990521,NaN,NaN,NaN,NaN,NaN
212,70686,226089,191526,871858,38932,288251.0,2101-08-11 00:27:00,2101-08-11 00:29:00,2.000000,ml,...,NaN,NaN,NaN,Hemoglobin,0.993007,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1826725,93321,225930,115396,3599677,44773,248745.0,2128-09-14 15:33:00,2128-09-14 18:43:00,31.666667,ml,...,NaN,9.033333,34.90,PTT,NaN,NaN,NaN,NaN,NaN,NaN
1826726,93321,225930,115396,3599677,44773,248745.0,2128-09-14 15:33:00,2128-09-14 18:43:00,31.666667,ml,...,NaN,9.033333,3.70,Potassium,NaN,NaN,NaN,NaN,NaN,NaN
1826727,93321,225930,115396,3599677,44773,248745.0,2128-09-14 15:33:00,2128-09-14 18:43:00,31.666667,ml,...,NaN,9.033333,1.00,Prothrombin time INR,NaN,NaN,NaN,NaN,NaN,NaN
1826728,93321,225930,115396,3599677,44773,248745.0,2128-09-14 15:33:00,2128-09-14 18:43:00,31.666667,ml,...,NaN,9.033333,2.53,Red blood cell,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
from scipy import stats
import numpy as np

In [55]:
res = stats.ttest_1samp(med_lab_pairs["ratio_(0, 6)_(0, 4)"].replace([np.inf, -np.inf], np.nan).dropna().to_numpy(), popmean=1)

In [56]:
res

TtestResult(statistic=1.0888925556761617, pvalue=0.27620205794950725, df=332795)

In [143]:
med_lab_pairs.groupby(["MED_NAME", "LAB_NAME"]).count()[["HADM_ID"]]

HADM_ID
MED_NAME               LAB_NAME                          
ACD-A Citrate (1000ml) Alanine aminotransferase        69
                       Albumin                         24
                       Amylase                         22
                       Asparate aminotransferase       69
                       Bilirubin                       71
...                                                   ...
Voriconazole           Potassium                       47
                       Prothrombin time INR            42
                       Red blood cell                  48
                       Sodium                          48
                       Uric Acid                        7

[5399 rows x 1 columns]

In [144]:
pairs_df = med_lab_pairs.groupby(["MED_NAME", "LAB_NAME"]).count()[["HADM_ID"]]
pairs = pairs_df[pairs_df["HADM_ID"]>100].index
discovery_res1 = []
for med_name, lab_name in pairs:
    stat_test_df = []
    for a_w in after_windows:
        for b_w in before_windows:
            vals = med_lab_pairs[med_lab_pairs["LAB_NAME"]==lab_name]
            vals = vals[vals["MED_NAME"]==med_name]
            vals = vals[f"ratio_{b_w}_{a_w}"].replace([np.inf, -np.inf], np.nan).dropna()
            if vals.shape[0]>0:
                res = stats.ttest_1samp(vals.to_numpy(), popmean=1)
                row = {
                    "Med Name": med_name,
                    "Lab Name": lab_name,
                    "Before Window (in Hours)": b_w,
                    "After Window (in Hours)": a_w,
                    "No. of Patients": vals.shape[0],
                    "1-Sampled Ttest" : res.pvalue
                }
            stat_test_df.append(row)
    if len(stat_test_df)>0:
        discovery_res1.extend(stat_test_df)
res_df1 = pd.DataFrame(discovery_res1)

/Users/pavan/Desktop/TAU/DrugLab/venv/lib/python3.9/site-packages/scipy/stats/_axis_nan_policy.py:502: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/Users/pavan/Desktop/TAU/DrugLab/venv/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1214: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/Users/pavan/Desktop/TAU/DrugLab/venv/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1214: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


KeyboardInterrupt: 

In [146]:
res_df1 = pd.DataFrame(discovery_res1)
res_df1

,Med Name,Lab Name,Before Window (in Hours),After Window (in Hours),No. of Patients,1-Sampled Ttest
0,ACD-A Citrate (1000ml),Calcium,"(0, 6)","(0, 4)",9,0.371663
1,ACD-A Citrate (1000ml),Calcium,"(6, 12)","(0, 4)",8,0.029424
2,ACD-A Citrate (1000ml),Calcium,"(0, 6)","(4, 8)",6,0.205565
3,ACD-A Citrate (1000ml),Calcium,"(6, 12)","(4, 8)",6,0.965059
4,ACD-A Citrate (1000ml),Calcium,"(0, 6)","(8, 12)",5,0.921116
...,...,...,...,...,...,...
9187,OR Platelet Intake,Magnesium,"(6, 12)","(0, 4)",15,0.720177
9188,OR Platelet Intake,Magnesium,"(0, 6)","(4, 8)",41,0.007541
9189,OR Platelet Intake,Magnesium,"(6, 12)","(4, 8)",15,0.132074
9190,OR Platelet Intake,Magnesium,"(0, 6)","(8, 12)",33,0.002722


In [155]:
new_pairs = set(pairs).difference(res_df1.set_index(["Med Name", "Lab Name"]).index)

In [173]:
pairs = new_pairs
discovery_res2 = []
for med_name, lab_name in pairs:
    stat_test_df = []
    vals1 = med_lab_pairs[med_lab_pairs["LAB_NAME"]==lab_name]
    vals1 = vals1[vals1["MED_NAME"]==med_name]
    for a_w in after_windows:
        for b_w in before_windows:
            vals = vals1[f"ratio_{b_w}_{a_w}"].replace([np.inf, -np.inf], np.nan).dropna()
            if vals.shape[0]>100:
                res = stats.ttest_1samp(vals.to_numpy(), popmean=1)
                row = {
                    "Med Name": med_name,
                    "Lab Name": lab_name,
                    "Before Window (in Hours)": b_w,
                    "After Window (in Hours)": a_w,
                    "No. of Patients": vals.shape[0],
                    "1-Sampled Ttest" : res.pvalue
                }
            stat_test_df.append(row)
    if len(stat_test_df)>0:
        discovery_res2.extend(stat_test_df)
res_df2 = pd.DataFrame(discovery_res2)

In [174]:
res_df = pd.concat([res_df1, res_df2])

In [175]:
res_df[res_df["No. of Patients"]>100]

,Med Name,Lab Name,Before Window (in Hours),After Window (in Hours),No. of Patients,1-Sampled Ttest
120,Acetaminophen-IV,Glucose,"(0, 6)","(0, 4)",180,0.384117
121,Acetaminophen-IV,Glucose,"(6, 12)","(0, 4)",182,0.273030
122,Acetaminophen-IV,Glucose,"(0, 6)","(4, 8)",174,0.724914
123,Acetaminophen-IV,Glucose,"(6, 12)","(4, 8)",168,0.720519
124,Acetaminophen-IV,Glucose,"(0, 6)","(8, 12)",114,0.708415
...,...,...,...,...,...,...
2869,OR Platelet Intake,Platelets,"(0, 6)","(0, 4)",119,0.000227
2870,OR Platelet Intake,Platelets,"(0, 6)","(0, 4)",119,0.000227
2871,OR Platelet Intake,Platelets,"(0, 6)","(0, 4)",119,0.000227
2872,OR Platelet Intake,Platelets,"(0, 6)","(0, 4)",119,0.000227


In [176]:
res_df[res_df["No. of Patients"]>100].to_csv(os.path.join(setup_io_config(root_path=root_path)[1], "pvals_mimic_non_vital_signs.csv"))

In [177]:
dis_analyzer = discovery.ClinicalDiscoveryAnalysis(med_lab_pairs)

In [178]:
test_pval_data, significant_hard_thres, significant_bonferroni, significant_fdr = dis_analyzer.generate_significant(res_df[res_df["No. of Patients"]>100].dropna(), statistical_test="1-Sampled Ttest")

In [179]:
significant_fdr

,Med Name,Lab Name,Before Window (in Hours),After Window (in Hours),No. of Patients,1-Sampled Ttest,BonferroniPvals,FDR Benjamini Corrected
444,Albumin 5%,Hematocrit,"(0, 6)","(0, 4)",289,5.160989e-19,3.221038e-01,4.601483e-04
445,Albumin 5%,Hematocrit,"(6, 12)","(0, 4)",341,1.553986e-06,1.000000e+00,1.974117e-01
446,Albumin 5%,Hematocrit,"(0, 6)","(4, 8)",171,3.436242e-06,1.000000e+00,5.758234e-01
450,Albumin 5%,Hemoglobin,"(0, 6)","(0, 4)",130,2.031604e-05,6.365503e-09,5.893984e-11
451,Albumin 5%,Hemoglobin,"(6, 12)","(0, 4)",202,1.971696e-03,3.687548e-08,2.973829e-10
...,...,...,...,...,...,...,...,...
2869,OR Platelet Intake,Platelets,"(0, 6)","(0, 4)",119,2.267149e-04,1.000000e+00,4.743351e-02
2870,OR Platelet Intake,Platelets,"(0, 6)","(0, 4)",119,2.267149e-04,1.000000e+00,4.743351e-02
2871,OR Platelet Intake,Platelets,"(0, 6)","(0, 4)",119,2.267149e-04,1.000000e+00,4.743351e-02
2872,OR Platelet Intake,Platelets,"(0, 6)","(0, 4)",119,2.267149e-04,1.000000e+00,4.743351e-02


In [180]:
merged_sig_pairs = pd.merge(pd.merge(significant_fdr, significant_bonferroni), significant_hard_thres)

In [183]:
merged_sig_pairs

,Med Name,Lab Name,Before Window (in Hours),After Window (in Hours),No. of Patients,1-Sampled Ttest,BonferroniPvals,FDR Benjamini Corrected
0,Albumin 5%,Hematocrit,"(0, 6)","(0, 4)",289,5.160989e-19,3.221038e-01,4.601483e-04
1,Albumin 5%,Hematocrit,"(6, 12)","(0, 4)",341,1.553986e-06,1.000000e+00,1.974117e-01
2,Albumin 5%,Lactic acid,"(0, 6)","(4, 8)",101,4.532815e-08,4.498439e-08,3.514405e-10
3,Amiodarone,Glucose,"(6, 12)","(0, 4)",233,2.718428e-09,1.000000e+00,6.905787e-01
4,Calcium Gluconate,Calcium,"(0, 6)","(4, 8)",271,4.877574e-17,8.335119e-25,3.087081e-26
...,...,...,...,...,...,...,...,...
1194,Piggyback,Lactic acid,"(0, 6)","(8, 12)",110,2.526701e-07,1.000000e+00,4.743351e-02
1195,Piggyback,Lactic acid,"(0, 6)","(8, 12)",110,2.526701e-07,1.000000e+00,4.743351e-02
1196,Piggyback,Lactic acid,"(0, 6)","(8, 12)",110,2.526701e-07,1.000000e+00,4.743351e-02
1197,Piggyback,Lactic acid,"(0, 6)","(8, 12)",110,2.526701e-07,1.000000e+00,4.743351e-02


In [184]:
merged_sig_pairs.to_csv(os.path.join(setup_io_config(root_path=root_path)[1], "sig_pairs_intersection_mimic_non_vital_signs.csv"))